# <font color = 'darkblue' size = 6> BHARAT DATA SCIENCE INTERN </font>
# <font color = 'brown' size = 5> TASK - 3 : STOCK PRICE PREDICTION BY USING LSTM </font>
# <font color = 'b' size = 3> NAME : KAVIARASU KARMEGAM </font>




<img src = stockk.jpg style = width:600;height:800>

# <font color = red size = 4> IMPORT LIBRARIES

In [3]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error

# <font color = 'red' size = 4 > READ THE DATASET </font>

In [6]:
data = pd.read_csv('Stock.csv')

In [7]:
data.head()

,Unnamed: 0,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,0,AAPL,2015-05-27 00:00:00+00:00,132.045,132.260,130.05,130.34,45833246,121.682558,121.880685,119.844118,120.111360,45833246,0.0,1.0
1,1,AAPL,2015-05-28 00:00:00+00:00,131.780,131.950,131.10,131.86,30733309,121.438354,121.595013,120.811718,121.512076,30733309,0.0,1.0
2,2,AAPL,2015-05-29 00:00:00+00:00,130.280,131.450,129.90,131.23,50884452,120.056069,121.134251,119.705890,120.931516,50884452,0.0,1.0
3,3,AAPL,2015-06-01 00:00:00+00:00,130.535,131.390,130.05,131.20,32112797,120.291057,121.078960,119.844118,120.903870,32112797,0.0,1.0
4,4,AAPL,2015-06-02 00:00:00+00:00,129.960,130.655,129.32,129.86,33667627,119.761181,120.401640,119.171406,119.669029,33667627,0.0,1.0


In [8]:
data1 = data.reset_index()['close']

In [9]:
data1

0       132.045
1       131.780
2       130.280
3       130.535
4       129.960
         ...   
1253    314.960
1254    313.140
1255    319.230
1256    316.850
1257    318.890
Name: close, Length: 1258, dtype: float64

In [10]:
data1.shape

(1258,)

In [1]:
plt.plot(data1 , color = "green")

NameError: name 'plt' is not defined

In [12]:
scaler = MinMaxScaler(feature_range=(0,1))

In [13]:
data1 = scaler.fit_transform(np.array(data1).reshape(-1,1))

In [14]:
data1

array([[0.17607447],
       [0.17495567],
       [0.16862282],
       ...,
       [0.96635143],
       [0.9563033 ],
       [0.96491598]])

# <font color = 'red' size = 4 > splitting the dataset into train and test split </font>

In [15]:
training_size = int(len(data1)*0.65)

In [16]:
test_size = len(data1) - training_size

In [17]:
train_data,test_data = data1[0:training_size,:],data1[training_size:len(data1),:1]

In [18]:
len(train_data)

817

In [19]:
len(test_data)

441

# <font color = 'red' size = 4 > converting the array values into a dataset matrix </font>

In [20]:

def create_dataset(dataset, time_step = 1):
    dataX , dataY = [], []
    for i in range(len(dataset) - time_step - 1 ):
        a = dataset[i:(i + time_step), 0]  
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [21]:
time_step = 100

In [22]:
X_train, y_train = create_dataset(train_data, time_step)

X_test, y_test = create_dataset(test_data, time_step)

In [23]:
X_train.shape

(716, 100)

In [24]:
y_train.shape

(716,)

In [25]:
X_test.shape

(340, 100)

In [26]:
y_test.shape

(340,)

In [27]:
# reshape the input to be in [samples, time steps, features] which is required for LSTM
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)

In [28]:
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

# <font color = 'red' size = 4 > CREATING THE STACKED LSTM MODEL <font/>

In [29]:
model=Sequential()

In [30]:
model.add(LSTM(50,return_sequences = True,input_shape = (100,1)))
model.add(LSTM(50,return_sequences = True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer = 'adam')

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           10400     
                                                                 
 lstm_1 (LSTM)               (None, 100, 50)           20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 50851 (198.64 KB)
Trainable params: 50851 (198.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 100,batch_size = 64,verbose = 1)

Epoch 1/100
12/12 [==============================] - 17s 555ms/step - loss: 0.0127 - val_loss: 0.0655
Epoch 2/100
12/12 [==============================] - 4s 347ms/step - loss: 0.0031 - val_loss: 0.0061
Epoch 3/100
12/12 [==============================] - 4s 336ms/step - loss: 0.0011 - val_loss: 0.0051
Epoch 4/100
12/12 [==============================] - 4s 339ms/step - loss: 8.1075e-04 - val_loss: 0.0041
Epoch 5/100
12/12 [==============================] - 4s 318ms/step - loss: 7.0973e-04 - val_loss: 0.0064
Epoch 6/100
12/12 [==============================] - 4s 317ms/step - loss: 6.5773e-04 - val_loss: 0.0063
Epoch 7/100
12/12 [==============================] - 4s 321ms/step - loss: 6.8310e-04 - val_loss: 0.0061
Epoch 8/100
12/12 [==============================] - 4s 313ms/step - loss: 6.4548e-04 - val_loss: 0.0054
Epoch 9/100
12/12 [==============================] - 4s 314ms/step - loss: 6.4938e-04 - val_loss: 0.0042
Epoch 10/100
12/12 [==============================] - 4s 315ms/ste

# <font color = 'blue' size = 4 > prediction and performance metrics </font>

In [33]:

train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

11/11 [==============================] - 1s 64ms/step


# <font color = 'blue' size = 4 > Back to original form </font>

In [34]:
 
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# <font color = 'blue' size = 4 > RMSE performance metrics (Training dataset) </font>

In [35]:

math.sqrt(mean_squared_error(y_train,train_predict))

142.75520158580665

# <font color = 'blue' size = 4 > RMSE performance metrics (Test dataset) </font>

In [ ]:

math.sqrt(mean_squared_error(y_test,test_predict))

# <font color = 'blue' size = 4 > shifting train predictions for plotting </font>

In [ ]:

look_back=100
trainPredictPlot = np.empty_like(data1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict

# <font color = 'blue' size = 4 > shifting test predictions for plotting </font>

In [ ]:
testPredictPlot = np.empty_like(data1)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(data1)-1, :] = test_predict

# <font color = 'blue' size = 4 > ploting baseline and predictions </font>

In [ ]:
plt.plot(scaler.inverse_transform(data1) , color = "yellow")
plt.plot(trainPredictPlot , color = "blue") # train predict data
plt.plot(testPredictPlot  , color = "red") # test predict data
plt.show()

# <font color = 'blue' size = 4 > previous 100 days </font>

In [ ]:
x_input = test_data[341:].reshape(1,-1)
x_input.shape

In [ ]:
temp_input = list(x_input)

In [ ]:
temp_input = temp_input[0].tolist()

In [ ]:
# <font color = 'blue' size = 4 > demonstrate prediction for next 30 days </font>

In [ ]:
#
lst_output=[]
n_steps=100
i=0
while(i<30):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input = np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        # print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        # print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    

print(lst_output)

In [ ]:
day_new=np.arange(1,101)

In [ ]:
day_pred=np.arange(101,131)

In [ ]:
plt.plot(day_new,scaler.inverse_transform(data1[1158:]) , color="green")
plt.plot(day_pred,scaler.inverse_transform(lst_output) , color = "orange")

In [ ]:
data3 = data1.tolist()
data3.extend(lst_output)
plt.plot(data3[1200:] , color = "green")